In [309]:
from collections import deque
import time
import math
from datetime import datetime, timedelta
import pandas as pd

In [310]:
#STEP 1: SPAWNING VISITORS, MAKE visitor_generator_df

spawning_df = pd.read_csv('../data/spawning.csv')

# Convert columns to the desired types
spawning_df['Average Wait Time'] = spawning_df['Average Wait Time'].astype(float)  # Convert to float
spawning_df['Number of Visitors'] = spawning_df['Number of Visitors'].astype(int)  # Convert to int
spawning_df['Number of New Visitors Arriving'] = spawning_df['Number of New Visitors Arriving'].astype(int)  # Convert to int

# print(spawning_df) # debugging

# Convert df into dictionary for easier reference
spawning = {}
for index, row in spawning_df.iterrows():
    number = row['Number of New Visitors Arriving']
    if number >= 0:
        spawning[row['Time']] = number
    else:
        spawning[row['Time']] = 0

print(spawning) # debugging

# take reference from Subgroup A for demand forecasting

{'10:00': 35, '10:05': 0, '10:10': 0, '10:15': 50, '10:20': 0, '10:25': 0, '10:30': 50, '10:35': 50, '10:40': 0, '10:45': 50, '10:50': 0, '10:55': 100, '11:00': 50, '11:05': 0, '11:10': 0, '11:15': 50, '11:20': 50, '11:25': 0, '11:30': 0, '11:35': 50, '11:40': 0, '11:45': 50, '11:50': 100, '11:55': 0, '12:00': 0, '12:05': 150, '12:10': 0, '12:15': 50, '12:20': 0, '12:25': 0, '12:30': 50, '12:35': 0, '12:40': 0, '12:45': 50, '12:50': 0, '12:55': 50, '13:00': 0, '13:05': 0, '13:10': 50, '13:15': 0, '13:20': 0, '13:25': 0, '13:30': 100, '13:35': 0, '13:40': 100, '13:45': 0, '13:50': 50, '13:55': 0, '14:00': 0, '14:05': 0, '14:10': 50, '14:15': 50, '14:20': 0, '14:25': 0, '14:30': 0, '14:35': 0, '14:40': 50, '14:45': 0, '14:50': 0, '14:55': 0, '15:00': 50, '15:05': 0, '15:10': 50, '15:15': 0, '15:20': 0, '15:25': 0, '15:30': 0, '15:35': 0, '15:40': 0, '15:45': 0, '15:50': 50, '15:55': 0, '16:00': 0, '16:05': 0, '16:10': 0, '16:15': 0, '16:20': 0, '16:25': 50, '16:30': 0, '16:35': 0, '16:40

In [ ]:
#STEP 2: IMPORT LIST OF ITINERARIES
itineraries = pd.read_csv('../data/itinerary_only_util.csv')
itineraries['path list'] = itineraries['path'].str.split()
# print(itineraries) # debugging

# Flatten all lists into a single list
all_possible_itineraries = [sublist for sublist in itineraries['path list']]

print(all_possible_itineraries) # debugging, each item in the flattened list is still a list

[['E', 'D', 'D', 'E', 'C'], ['C', 'D'], ['E', 'E'], ['C', 'E', 'D'], ['E', 'C', 'D'], ['E', 'C', 'D'], ['E', 'C', 'D'], ['C', 'D'], ['E', 'C', 'D'], ['E', 'E', 'C', 'D'], ['D', 'C', 'E', 'C', 'E'], ['C', 'E'], ['C', 'E', 'D'], ['C', 'E', 'D'], ['C', 'D', 'E'], ['C', 'E', 'D', 'G'], ['E'], ['E', 'C', 'D'], ['E', 'C', 'D']]
[['E', 'D', 'D', 'E', 'C'], ['C', 'D'], ['E', 'E'], ['C', 'E', 'D'], ['E', 'C', 'D'], ['E', 'C', 'D'], ['E', 'C', 'D'], ['C', 'D'], ['E', 'C', 'D'], ['E', 'E', 'C', 'D'], ['D', 'C', 'E', 'C', 'E'], ['C', 'E'], ['C', 'E', 'D'], ['C', 'E', 'D'], ['C', 'D', 'E'], ['C', 'E', 'D', 'G'], ['E'], ['E', 'C', 'D'], ['E', 'C', 'D']]


In [312]:
class Visitor: # generate unique Visitor
    def __init__(self, id, itinerary, fast_pass: int, arrival_time, current_action):
        """
        :param id: a unique ID
        :param itinerary: a list of itinerary (where they will go, in sequence)
        :param fast_pass: 0 for regular ticket, 1 for fast-pass ticket
        """
        self.id = id
        self.fast_pass = fast_pass 
        self.itinerary = itinerary 
        self.current_time = arrival_time # First set to when they are 'spawned'
        self.accumulated_waiting_time = datetime(1900, 1, 1, 0, 0, 0)
        self.accumulated_satisfactory_score = 0 ### LINK TO JON's part
        
        self.current_action = current_action
        self.next_action = current_action
        self.current_ride_start = None # The moment they get served from the queue
        
        self.service_start_time = None
        self.service_arrival_time = None

        self.completed = False

    def __repr__(self): # representation method 
        return f"Visitor {self.id, self.fast_pass}"
    
    def travel(self, distance): # visitor travels between nodes ### link to Jon's
        """
        :param distance: distance between two position
        """
        time_travelled = distance # It takes 1 minute to travel distance = 1
        self.current_time += timedelta(minutes=int(time_travelled))


In [313]:
attraction_generator_df = pd.read_csv('../data/theme_park_nodes.csv')


In [314]:
def visitor_next_step(person:Visitor):

    if person.itinerary:
        choice_index = person.itinerary.pop(0)
        choices = attraction_generator_df[attraction_generator_df['index'] == choice_index]['name']
        if not choices.tolist():
            print(choice_index) # debugging
        person.next_action = random.choices(choices.tolist(),k=1)[0] # because random.choices() return a list, therefore need index
    # else:
    #     person.completed = True
    #     #person.next_action = 'Entrance'
    #     print(f"{person} has completed their itinerary at {person.current_time}.")
    


def visitor_moving(person:Visitor):
    #if person.completed == False:
    # Filter paths_df to find the matching rows
    matching_rows = paths_df[(paths_df['source'] == person.current_action) & (paths_df['target'] == person.next_action)]

    # Check if the filtered DataFrame is empty
    if not matching_rows.empty:
        # Access the 'distance' value from the first matching row
        distance = matching_rows['distance'].iloc[0]
    else:
        # Handle the case where no match is found
        print('Error: No matching path found for source',person.current_action ,'and target', person.next_action)
        distance = None  # or some default value
    person.travel(distance) # The visitor travel to that attraction/utility



def add_visitor_to_holding_queue(person:Visitor):
    #if person.completed == False:
    key = person.next_action
    if key in attractions_dict.keys():
        item = attractions_dict[key]
        item.holding_visitors.append(person)
    elif key in seasonals_dict.keys():
        item = seasonals_dict[key]
        item.holding_visitors.append(person)
    elif key in utilities_dict.keys():
        item = utilities_dict[key]
        item.holding_visitors.append(person)
    

In [315]:
class Attraction: # Including shows, rides
    def __init__(self, name: str, ride_duration: int, ride_capacity: int):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.holding_visitors = deque() # To store visitors before adding them to regular/fast-pass queue
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_duration = int(ride_duration)  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.current_time = 10  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
    def __repr__(self): # representation method 
        return f"{self.name}"

    def get_rides_served(self):
        return self.rides_served

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.current_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue for {self} at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue for {self} at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at {self} at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(int(min(self.ride_capacity//2, len(self.fast_pass_queue)))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue for {self} is being served.") # for debugging
                    self.total_served += 1
                    print(f"{self} 's total serving so far: {self.total_served}")  # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(int(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue)))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue for {self} is being served.") # for debugging
                    self.total_served += 1
                    print(f"{self} 's total serving so far: {self.total_served}")  # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print(f"No one is waiting in the queues for {self} at time", self.current_time)
        
        self.rides_served += 1
        print(f"Number of rides so far for {self} is{self.rides_served}") # debugging

        # The time now is right before the ride starts. We can update visitor's current time to calculate waiting time
        for visitor in riding_visitors:
            visitor.current_ride_start = self.current_time
            wait_time = visitor.current_ride_start - visitor.current_time # wait time = time the ride start - time visitor arrive at the queue
            print(f"{visitor} waited for {self} for {wait_time}") # debugging
            visitor.accumulated_waiting_time += timedelta(minutes=wait_time.total_seconds() // 60)
            # Then, update the time for Visitor objects 
            visitor.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration)
            # and we send them off to the next item on their itinerary:
            visitor.current_action = visitor.next_action
            visitor_next_step(visitor)
            visitor_moving(visitor)
            add_visitor_to_holding_queue(visitor)

        
        # Lastly, we update the time of this Attraction object:
        self.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration)
        print(f"The time at {self.name} now is {self.current_time}.")  

    def get_data(self): # Note: should only be called AFTER calling process_queue()
        if math.isnan(self.ride_duration):
            self.ride_duration = 0
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, crowd_level, curr_wait_time
    
    

In [316]:
class Seasonal: # Including shows, rides, has fixed timing
    def __init__(self, name: str, ride_duration: int, ride_capacity: int, timeslot):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        :param timeslot: schedule of the seasonal attraction, list
        """
        self.name = name
        self.holding_visitors = deque() # To store visitors before adding them to regular/fast-pass queue
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_duration = int(ride_duration)  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.timeslot = timeslot
        self.current_time = timeslot.pop(0) 
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
    def __repr__(self): # representation method 
        return f"{self.name}"

    def get_rides_served(self):
        return self.rides_served

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.current_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue for{self} at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue for{self} at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at{self} at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(int(min(self.ride_capacity//2, len(self.fast_pass_queue)))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue for {self} is being served.") # for debugging
                    self.total_served += 1
                    print(f"{self} 's total serving so far: {self.total_served}") # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue for {self} is being served.") # for debugging
                    self.total_served += 1
                    print(f"{self} 's total serving so far: {self.total_served}") # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print("No one is waiting in the queues at time", self.current_time)
        
        self.rides_served += 1
        print(f"Number of rides so far is{self.rides_served}") # debugging

        # The time now is right before the ride starts. We can update visitor's current time to calculate waiting time
        for visitor in riding_visitors:
            visitor.current_ride_start = self.current_time
            wait_time = visitor.current_ride_start - visitor.current_time # wait time = time the ride start - time visitor arrive at the queue
            print(f"{visitor} waited for {self} for {wait_time}") # debugging
            visitor.accumulated_waiting_time += timedelta(minutes=wait_time.total_seconds() // 60)
            # Then, update the time for Visitor objects 
            visitor.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration) 
            # and we send them off to the next item on their itinerary:
            visitor.current_action = visitor.next_action
            visitor_next_step(visitor)
            visitor_moving(visitor)
            add_visitor_to_holding_queue(visitor)

        
        # Lastly, we update the time of this Attraction object:
        if self.timeslot:
            self.current_time = self.timeslot.pop(0)
        print(f"The time at {self.name} now is {self.current_time}.")  

    def get_data(self): # Note: should only be called AFTER calling process_queue()
        if math.isnan(self.ride_duration):
            self.ride_duration = 0
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, crowd_level, curr_wait_time
    

In [317]:
class Utility: # Including toilets, dining outlets, souvenir shops
    def __init__(self, name: str, service_duration: int, util_capacity: int):
        """
        Initialize a utility in the part.
        :param name: Name of the utility.
        :param service_time: Duration spent inside the Utility
        :param util_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.service_duration = int(service_duration)
        self.util_capacity = util_capacity 
        self.holding_visitors = deque()
        self.queue = deque()
        self.serving_visitors = deque() # To track visitors being served at the current moment
        self.current_time = None
        self.total_served = 0  # cumulative number of served visitors 
        self.most_current_visitor_start_time = None # The moment the most recent visitor start using the utility

    def __repr__(self): # representation method 
        return f"{self.name}"
    

    def process_inside(self):
        for _ in range (int(self.util_capacity)):
            if self.serving_visitors:
                visitor = self.serving_visitors.popleft()
                if visitor.service_start_time + timedelta(minutes=self.service_duration) <= self.current_time:
                    visitor.current_time = self.current_time
                    
                    # Calculate and accumulate waiting time for that visitor
                    wait_time = (visitor.service_start_time - visitor.service_arrival_time).total_seconds() // 60
                    visitor.accumulated_waiting_time += timedelta(minutes=wait_time)

                    # and we send them off to the next item on their itinerary:
                    visitor.current_action = visitor.next_action
                    visitor_next_step(visitor)
                    visitor_moving(visitor)
                    add_visitor_to_holding_queue(visitor)
                    print(f"{visitor} has completed their time in {self.name} and is leaving.")
                else:
                    self.serving_visitors.append(visitor)


    def process_queue(self):
        while len(self.serving_visitors) < self.util_capacity and self.queue:
            visitor = self.queue.popleft()
            self.serving_visitors.append(visitor)
            visitor.service_start_time = self.current_time

    def add_visitor(self):
        while self.holding_visitors:
            visitor = self.holding_visitors.popleft()

            if len(self.serving_visitors) < self.util_capacity: # have vacancy
                self.serving_visitors.append(visitor)
                visitor.service_start_time = self.current_time
                visitor.service_arrival_time = self.current_time
            else:
                self.queue.append(visitor)
                visitor.service_arrival_time = self.current_time
    

    def advance_time(self, minutes: int):
        """
        Advances the current time by a specified number of minutes.
        """
        self.current_time += timedelta(minutes=minutes)
        print(f"The time now at {self.name} is {self.current_time}.")

    def get_data(self):
        """
        Returns the current crowd level in the queue and 
        returns the estimated wait time based on the current crowd level and service duration.
        """
        if math.isnan(self.service_duration):
            self.service_duration = 0
        crowd_level = len(self.queue)
        print(f"At {self.current_time}, the crowd level in {self.name} is {crowd_level}")
        curr_wait_time = math.ceil(crowd_level / self.util_capacity) * self.service_duration
        print(f"At {self.current_time}, the estimated wait time in {self.name} is {curr_wait_time} minutes")
        return self.current_time, crowd_level, curr_wait_time
    
    def get_visitors_served(self):
        return self.total_served


In [318]:
import random

class ThemePark:
    def __init__(self, spawning_dict, all_possible_itineraries):
        """
        :param spawning_dict: a dictionary containing number of new visitors for each time slot

        """
        self.spawning_dict = spawning_dict
        self.all_possible_itineraries = all_possible_itineraries
        self.global_time = None  # Overall time counter for the simulation, take from the time_slot object
        self.attractions = []
        self.utilities = []
        self.seasonals = []
        self.existing_visitors = {} # A dictionary to store the lists of spawning visitors at each time increments
        self.visitors_count = 0 # to track and create new unique visitors

    def add_attraction(self, attraction: Attraction):
        """
        Add an attraction to the park.
        :param attraction: Attraction object.
        """
        self.attractions.append(attraction)

    def add_utility(self, utility: Utility):
        """
        Add a utility to the park.
        :param utility: Utility object.
        """
        self.utilities.append(utility)

    def add_seasonal(self, seasonal: Seasonal):
        """
        Add a seasonal attraction to the park.
        :param seasonal: Seasonal object.
        """
        self.seasonals.append(seasonal)

    def spawn_visitor(self, time):
        """ 
        :param time: time slot
        """
        visitors = [] # List to store Visitor objects
        for _ in range(int(self.spawning_dict[time.strftime('%H:%M')])):
            new_visitor = Visitor(id= self.visitors_count, 
                                    itinerary=random.choice(self.all_possible_itineraries), 
                                    fast_pass=random.choices([0,1], weights=[0.8, 0.2],k=1)[0], # 20% of purchasing express pass
                                    arrival_time=time,
                                    current_action='Entrance')
            self.visitors_count += 1
            visitors.append(new_visitor)
        return visitors
    
    def open_park(self, time): 
        """ 
        :param time: 
        """
        for item in self.attractions+self.utilities+self.seasonals:
            item.current_time = time
        # for utility in :
        #     utility.current_time = time
        # for seasonal in :
        #     seasonal

    def advance_time(self, minutes: int):
        """
        Advances the current time by a specified number of minutes.
        """
        self.global_time += timedelta(minutes=minutes)
        print(f"The time now is {self.global_time}.")



In [319]:
#STEP 3: CREATING NODES, MAKE NODE LIST 

attractions = [] # List to store Attraction objects
utilities = []
seasonals = []


for index, row in attraction_generator_df.iterrows():
    if row['type'] == 'Ride':
        new_attraction = Attraction(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity']) # change according to the finalised csv
        attractions.append(new_attraction) 
    if row['type'] == 'Seasonal':

        time_string = row['timeslots']
        time_list = time_string.split(", ") 
        datetime_list = [datetime.strptime(item, "%H:%M") for item in time_list]


        print(datetime_list)
        new_seasonal = Seasonal(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity'], timeslot=datetime_list) # change according to the finalised csv
        seasonals.append(new_seasonal)     
    if row['type'] in ['Restroom', 'Dining Outlet', 'Food Cart', 'Retail']:
        new_utility = Utility(name=row['name'], service_duration=row['duration'], util_capacity=row['capacity'])
        utilities.append(new_utility)

attractions_dict = {item.name: item for item in attractions}
utilities_dict = {item.name: item for item in utilities}
seasonals_dict = {item.name: item for item in seasonals}


print(attractions) # debugging
print(utilities) # debugging
print(seasonals) # debugging

print(attraction_generator_df[attraction_generator_df['index'] == 'A']['name']) # debugging
print(type(attraction_generator_df[attraction_generator_df['index'] == 'A']['name'])) # debugging


[datetime.datetime(1900, 1, 1, 10, 35), datetime.datetime(1900, 1, 1, 13, 0), datetime.datetime(1900, 1, 1, 15, 0), datetime.datetime(1900, 1, 1, 17, 55)]
[datetime.datetime(1900, 1, 1, 10, 5), datetime.datetime(1900, 1, 1, 11, 55), datetime.datetime(1900, 1, 1, 13, 55), datetime.datetime(1900, 1, 1, 15, 55), datetime.datetime(1900, 1, 1, 17, 25)]
[datetime.datetime(1900, 1, 1, 10, 15), datetime.datetime(1900, 1, 1, 12, 25), datetime.datetime(1900, 1, 1, 14, 5), datetime.datetime(1900, 1, 1, 15, 50)]
[datetime.datetime(1900, 1, 1, 11, 30), datetime.datetime(1900, 1, 1, 13, 30), datetime.datetime(1900, 1, 1, 15, 30), datetime.datetime(1900, 1, 1, 18, 20)]
[datetime.datetime(1900, 1, 1, 12, 45), datetime.datetime(1900, 1, 1, 15, 0), datetime.datetime(1900, 1, 1, 17, 15)]
[datetime.datetime(1900, 1, 1, 12, 0), datetime.datetime(1900, 1, 1, 13, 0), datetime.datetime(1900, 1, 1, 14, 0), datetime.datetime(1900, 1, 1, 15, 0), datetime.datetime(1900, 1, 1, 16, 0), datetime.datetime(1900, 1, 1,

In [320]:
#STEP 4: CREATING USS, PUT THINGS TOGETHER

# Creating ThemePark object
USS = ThemePark(spawning, all_possible_itineraries)

# Add Attraction objects to the park
for item in attractions:
    USS.add_attraction(item)

# Add Utility objects to the park
for item in utilities:
    USS.add_utility(item)

# Add Seasonal objects to the park
for item in seasonals:
    USS.add_seasonal(item)

# Open the park at 10:00, update time of all attractions and utilities
USS.global_time = datetime.strptime("10:00", "%H:%M")
USS.open_park(USS.global_time)

# Create a dictionary to store data of waiting time and crowd level across attractions and utilities across the theme park
park_data = {}

In [321]:
#STEP 5: IMPORTING EDGES (to facilitate Visitors travelling from one node to another)
paths_df = pd.read_csv('../data/theme_park_paths.csv')

In [322]:
print(USS.spawning_dict.keys())

dict_keys(['10:00', '10:05', '10:10', '10:15', '10:20', '10:25', '10:30', '10:35', '10:40', '10:45', '10:50', '10:55', '11:00', '11:05', '11:10', '11:15', '11:20', '11:25', '11:30', '11:35', '11:40', '11:45', '11:50', '11:55', '12:00', '12:05', '12:10', '12:15', '12:20', '12:25', '12:30', '12:35', '12:40', '12:45', '12:50', '12:55', '13:00', '13:05', '13:10', '13:15', '13:20', '13:25', '13:30', '13:35', '13:40', '13:45', '13:50', '13:55', '14:00', '14:05', '14:10', '14:15', '14:20', '14:25', '14:30', '14:35', '14:40', '14:45', '14:50', '14:55', '15:00', '15:05', '15:10', '15:15', '15:20', '15:25', '15:30', '15:35', '15:40', '15:45', '15:50', '15:55', '16:00', '16:05', '16:10', '16:15', '16:20', '16:25', '16:30', '16:35', '16:40', '16:45', '16:50', '16:55', '17:00', '17:05', '17:10', '17:15', '17:20', '17:25', '17:30', '17:35', '17:40', '17:45', '17:50', '17:55', '18:00', '18:05', '18:10', '18:15', '18:20', '18:25', '18:30', '18:35', '18:40', '18:45', '18:50', '18:55', '19:00', '19:05',

In [323]:
       
  #STEP 6: UP AND RUNNING

 # Create Visitors objects
# while USS.global_time <= datetime.strptime("12:00", "%H:%M"): 
    # if USS.global_time in USS.spawning_dict.keys():
USS.existing_visitors[USS.global_time] = USS.spawn_visitor(USS.global_time) 
    # Update USS global time, ready for the next iteration:
    
for person in USS.existing_visitors[USS.global_time]:
    visitor_next_step(person)
    visitor_moving(person)
    add_visitor_to_holding_queue(person)

# USS.advance_time(5)    
for i in range (1,32):
    print(USS.existing_visitors[USS.global_time][i].itinerary)

['D']
[]
[]
[]
['D']
[]
[]
[]
['E']
['C', 'D']
['D']
['D']
[]
[]
[]
[]
[]
['D']
['D']
['E']
['C', 'D']
[]
[]
[]
['D']
['E', 'D', 'G']
[]
[]
['D', 'D', 'E', 'C']
['D']
['C', 'E', 'C', 'E']


In [324]:



for item in USS.attractions:
    while item.current_time <= datetime.strptime("12:00", "%H:%M"): # iterate until before closing time    
        for visitor in item.holding_visitors:
            item.add_visitor(visitor)
        item.process_queue()
        time_slot, crowd, wait = item.get_data() # Note: should only be called AFTER calling process_queue()
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait
    


while USS.utilities[0].current_time <= datetime.strptime("12:00", "%H:%M"): # iterate until before closing time  
    for item in USS.utilities:
        item.process_inside()
        item.process_queue()
        item.add_visitor()
        time_slot, crowd, wait = item.get_data() 
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait
        # Update current time for the utility
        item.advance_time(1)

for item in USS.seasonals:
    while item.current_time <= datetime.strptime("12:00", "%H:%M"): # iterate until before closing time  
        for visitor in item.holding_visitors:
            item.add_visitor(visitor)
        item.process_queue()
        time_slot, crowd, wait = item.get_data() # Note: should only be called AFTER calling process_queue()
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait



No one is waiting in the queues for Accelerator at time 1900-01-01 10:00:00
Number of rides so far for Accelerator is1
The time at Accelerator now is 1900-01-01 10:03:00.
No one is waiting in the queues for Accelerator at time 1900-01-01 10:03:00
Number of rides so far for Accelerator is2
The time at Accelerator now is 1900-01-01 10:06:00.
No one is waiting in the queues for Accelerator at time 1900-01-01 10:06:00
Number of rides so far for Accelerator is3
The time at Accelerator now is 1900-01-01 10:09:00.
No one is waiting in the queues for Accelerator at time 1900-01-01 10:09:00
Number of rides so far for Accelerator is4
The time at Accelerator now is 1900-01-01 10:12:00.
No one is waiting in the queues for Accelerator at time 1900-01-01 10:12:00
Number of rides so far for Accelerator is5
The time at Accelerator now is 1900-01-01 10:15:00.
No one is waiting in the queues for Accelerator at time 1900-01-01 10:15:00
Number of rides so far for Accelerator is6
The time at Accelerator no

In [325]:
# Upon finishing a time period, data on number of visitors served can be retrieved
quantity_data = {}

for item in USS.attractions:
    quantity_data[item.name] = item.total_served
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging

for item in USS.utilities:
    quantity_data[item.name] = item.get_visitors_served()   
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging

for item in USS.seasonals:
    quantity_data[item.name] = item.total_served
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging



print(park_data['Revenge of the Mummy']) # debugging

Accelerator has served  0  visitors.
Battlestar Galactica: CYLON has served  0  visitors.
Battlestar Galactica: HUMAN has served  0  visitors.
Canopy Flyer has served  0  visitors.
Dino Soarin' has served  0  visitors.
Enchanted Airways has served  0  visitors.
Jurassic Park Rapids Adventure has served  0  visitors.
Lights Camera Action Hosted by Steven Spielberg has served  0  visitors.
Magic Potion Spin has served  0  visitors.
Puss In Boots Giant Journey has served  0  visitors.
Revenge of the Mummy has served  0  visitors.
Sesame Street Spaghetti Space Chase has served  0  visitors.
Shrek 4D Adventure has served  0  visitors.
TRANSFORMERS The Ride: The Ultimate 3D Battle has served  0  visitors.
Treasure Hunters has served  0  visitors.
Starbucks has served  0  visitors.
Mel's Drive-In has served  0  visitors.
KT's Grill has served  0  visitors.
Loui's NY Pizza Parlor has served  0  visitors.
StarBot Cafe has served  0  visitors.
Oasis Spice Cafe has served  0  visitors.
Discovery 

In [326]:
import pandas as pd
import os
# Dictionary to store DataFrames with dynamic names
dataframes = {}

# Iterate over each node (e.g., "Accelerator")
for node, time_slot in park_data.items():
    # Initialize a list to hold rows of data for each time slot
    rows = []
    
    # Iterate over each time entry in the node's data
    for timestamp, entry in time_slot.items():
        if isinstance(entry, dict):  # Ensure entry is a dictionary (which it is)
            # Prepare a row dictionary combining the timestamp and its data
            row = {'timestamp': timestamp}
            
            # Check if 'crowd level' and 'waiting time' are integers or dictionaries
            if isinstance(entry['crowd level'], dict):
                row['fast_pass_crowd_level'] = entry['crowd level']['fast_pass_queue']
                row['regular_crowd_level'] = entry['crowd level']['regular_queue']
            else:
                # Assuming 'crowd level' is just an integer (e.g., 0)
                row['fast_pass_crowd_level'] = entry['crowd level']
                row['regular_crowd_level'] = entry['crowd level']

            if isinstance(entry['waiting time'], dict):
                row['fast_pass_waiting_time'] = entry['waiting time']['fast_pass_queue']
                row['regular_waiting_time'] = entry['waiting time']['regular_queue']
            else:
                # Assuming 'waiting time' is just an integer (e.g., 0)
                row['fast_pass_waiting_time'] = entry['waiting time']
                row['regular_waiting_time'] = entry['waiting time']
            
            rows.append(row)

    # Create DataFrame for each node
    df = pd.DataFrame(rows)
    
    # Create a dynamic key for the dictionary using the node name
    df_name = f'{node}_df'
    
    # Store the DataFrame in the dictionary
    dataframes[df_name] = df

# Get current time for appending to folder name
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Create output directory with timestamp included in folder name
output_dir = f'../data/simulation_output_{current_time}/'
os.makedirs(output_dir, exist_ok=True)

# Export each DataFrame to a CSV file
for key, df in dataframes.items():
    csv_file = os.path.join(output_dir, f'{key}.csv')
    df.to_csv(csv_file, index=False)

